# Unique continuation for an elliptic interface problem 

In this notebook we revisit the elliptic interface problem treated in [cutfem.ipynb](cutfem.ipynb) in a slightly more challenging setting. Let us first recall the basic PDE problem: 
 
 $$
\left\{
\begin{aligned}
          \mathcal{L} u = & \, f 
          & & \text{in}~~ \Omega_{\pm}, 
          \\
          [\![u]\!] = & \, 0 
          &  & \text{on}~~ \Gamma, 
          \\
          [\![-\mu \nabla u \cdot \mathbf{n}]\!]   = & \, 0 
          &  & \text{on}~~ \Gamma,
        \end{aligned} \right.
$$
 
for $\mathcal{L}u_{\pm} := -\nabla \cdot (\mu_{\pm} \nabla u_{\pm})$ defined in subdomains $\Omega_{\pm}$. If you are eagle eyed, you may have noticed that we omitted the boundary conditions on $\partial \Omega$ which in [cutfem.ipynb](cutfem.ipynb) were given by $u =  u_D$ for some given function $u_D$ on $\partial \Omega$. This is not a mistake. We will indeed assume here that boundary data is unknown which renders this problem ill-posed. 

<div>
<img src="graphics/uc-overview-notebook.png" width="600"/>
</div>

To recover uniqueness and a limited form of stability we instead assume that measurements $q=u|_{\omega}$ in a subset $\omega \subset \Omega_{-}$ are available. Our objective is then to continue / extend the solution to a larger subset $\omega \subset B $ across the interface by solving a PDE constrained optimization problem. To this end, the basic CutFEM discretization from [cutfem.ipynb](cutfem.ipynb) has to be complemented by suitable regularization terms which are required to deal with ill-posed problems of this form. While we focus here on implementational aspects, we refer to the preprint [arXiv:2307.05210](https://arxiv.org/pdf/2307.05210.pdf) for mathematical details and error analysis.

## The usual imports

In [1]:
from netgen.geom2d import CSG2d, Rectangle
from ngsolve import *
from xfem import *
from xfem.lsetcurv import LevelSetMeshAdaptation
from math import pi
from netgen.csg import *
import numpy as np

importing ngsxfem-2.1.dev


## Mesh 
For simplicity we create a mesh in which the subdomains $\omega= [-0.5,0.5]^2$ and $B=[-1.25,1.25]^2$ are fitted. This is easy to achieve using constructive solid geometry definitions, see [CSG in 2D](https://docu.ngsolve.org/latest/i-tutorials/unit-4.1.2-csg2d/csg2d.html). 

In [2]:
geo = CSG2d()
omega_dom = Rectangle( pmin=(-0.5,-0.5), pmax=(0.5,0.5), mat="omega", bc="bc_omega")
B_dom = Rectangle( pmin=(-1.25,-1.25), pmax=(1.25,1.25), mat="B", bc="bc_B")
full_dom = Rectangle( pmin=(-1.5,-1.5), pmax=(1.5,1.5), mat="full", bc="bc_Omega")
only_B = B_dom - omega_dom
only_B.Mat("only_B")
rest = full_dom - B_dom
geo.Add(omega_dom)
geo.Add(only_B)
geo.Add(rest)
mesh = Mesh(geo.GenerateMesh(maxh=0.125))

Here, `mesh.Materials("omega")` correspond to $\omega$ and `mesh.Materials("only_B|omega")` to the target domain $B$.

## Levelset 
The unit ball in the $4$-norm will be used as the levelset function. Note that the subdomains $\omega$ and $B$ are clearly visible in the webgui below. 

In [3]:
r44 = x**4 + y**4
r41 = sqrt(sqrt(r44))
levelset = r41 - 1.0
DrawDC(levelset, -3.5, 2.5, mesh,"levelset")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Sample solution
We chose the diffusivities as $\mu_{-} = 2, \mu_{+} = 20$ and calculate the right hand side so that `solution` solves the PDE problem (it is easy to check that the interface conditions are fulfilled as well).

In [4]:
mu = (2,20)
solution = [(1/sqrt(2.0))*(1+pi*mu[0]/mu[1]) - cos(pi / 4 * r44), (mu[0]/mu[1])*(pi/sqrt(2.0)) * r41]
coef_f = [-mu[i] * (solution[i].Diff(x).Diff(x) + solution[i].Diff(y).Diff(y) ) for i in range(2)]

## Variational formulation and discretization space

The variational formulation to approximate the solution $u$ is given as follows:
Find $(u_h,z_h) \in V_h^\Gamma \times V_h^0$ such that 
$$
A_h(v_h,z_h) + s_h(u_h,v_h) + (u_h-q,v_h)_{ \omega } + A_h(u_h,w_h) - s_h^{\ast}(z_h,w_h) = \ell(v_h,w_h)  \qquad (*)
$$ 
for all $(v_h,w_h) \in V_h^\Gamma \times V_h^0$. Here, 

* $u_h$ is a discretization of $u$ sought in the CutFEM space $V_h^\Gamma$ introduced in [cutfem.ipynb](cutfem.ipynb),
* $z_h$ is a Lagrange multiplier which lives in a standard finite element space $V_h^0$ with homogeneous boundary conditions on $\partial \Omega$. 

Before we go into more detail on the terms in the variational formulation, let us first set up our discretization space $W = V_h^\Gamma \times V_h^0$.

In [5]:
order = 2 
lsetadap = LevelSetMeshAdaptation(mesh, order=order, levelset=levelset)
lsetp1 = lsetadap.lset_p1
ci = CutInfo(mesh, lsetp1)
Vh = H1(mesh, order=order, dirichlet=[],dgjumps=True)
Vh0 = H1(mesh, order=order, dirichlet="bc_Omega",dgjumps=False)
hasneg = ci.GetElementsOfType(HASNEG)
haspos = ci.GetElementsOfType(HASPOS)
hasif = ci.GetElementsOfType(IF)
Vh_Gamma = Compress(Vh, GetDofsOfElements(Vh, hasneg)) \
              * Compress(Vh, GetDofsOfElements(Vh, haspos)) \
              * Vh0

We obtain test and trial functions in the usual way. Note that the dual variable is in fact univalued on $\Omega$ but we prefer to define `z = [z0,z0]` for implementational / notational ease. 

In [ ]:
u1,u2,z0 =  Vh_Gamma.TrialFunction()
v1,v2,w0 =  Vh_Gamma.TestFunction()
u = [u1,u2]
z = [z0,z0]
v = [v1,v2]
w = [w0,w0]
gradu, gradz, gradv, gradw = [[grad(fun[i]) for i in [0, 1]] for fun in [u, z, v, w]]

## Unfitted Nitsche formulation

Let us now elaborate on the terms in the variational formulation (*). The terms $A_h(u_h,w_h)$ and $A_h(v_h,z_h) $  simply correspond to the standard unfitted Nitsche formulation. Since the dual variable is continuous only the adjoint consistency term remains on the interface, i.e. we have

$$
A_h(u,w) = \sum\limits_{\pm} (\mu_{\pm} \nabla u_{\pm}, \nabla w_{\pm})_{ \Omega_{{\pm},h} } + \int\limits_{\Gamma_h}  \{\!\!\{ -\mu \nabla w  \} \!\!\}  \cdot \mathbf{n} [\![u]\!].
$$

We also define the corresponding right hand side. The implementation given below is well-known from [cutfem.ipynb](cutfem.ipynb).

In [ ]:
dC = tuple([dCut(lsetp1, dt, deformation=lsetadap.deform,
                     definedonelements=ci.GetElementsOfType(HAS(dt)))
                for dt in [NEG, POS]])
ds = dCut(lsetp1, IF, deformation=lsetadap.deform)

n = 1.0 / grad(lsetp1).Norm() * grad(lsetp1)
kappa = ( mu[1]/sum(mu) , mu[0]/sum(mu) )
average_flux_z = sum([- kappa[i] * mu[i] * gradz[i] * n for i in [0, 1]])
average_flux_w = sum([- kappa[i] * mu[i] * gradw[i] * n for i in [0, 1]])

# a(uh,wh)
a = BilinearForm(Vh_Gamma, symmetric=True)
a += sum(mu[i] * gradu[i] * gradw[i] * dC[i] for i in [0, 1])
a +=  average_flux_w * (u[0] - u[1]) * ds

# a(vh,zh)
a += sum(mu[i] * gradv[i] * gradz[i] * dC[i] for i in [0, 1])
a +=  average_flux_z * (v[0] - v[1]) * ds

f = LinearForm(Vh_Gamma)
f += sum(coef_f[i] * w[i] * dC[i] for i in [0, 1])

## Data constraint 

Next we add the data constraint  $(u_h-q,v_h)_{ \omega }$ to the bilinear form and the right hand side. 

In [ ]:
gamma_data = 1e5
a += sum( gamma_data * u[i] * v[i] * dCut(lsetp1, dt,definedon=mesh.Materials("omega"), deformation=lsetadap.deform)
            for i,dt in zip([0, 1],[NEG,POS]) )
f += sum(gamma_data * solution[i] * v[i] * dCut(lsetp1, dt,definedon=mesh.Materials("omega"), deformation=lsetadap.deform)
            for i,dt in zip([0, 1],[NEG,POS]) )

## Primal stabilization terms
The main task is to definine the terms of the primal stabilization $s_h(u_h,v_h)$. These terms can be interpreted as regularizers which are used to incorporate a priori knowledge of the solution.

## Continuous interior penalty 
The first term is the well-known continuous interior penalty (CIP)

$$
J_{ \mathrm{CIP} }(u,v) := \sum\limits_{ \pm } \sum\limits_{ F \in  \mathcal{F}^{\pm} } h \!\! \int\limits_{ F } \!\! \mu_{\pm} [\![ \nabla u_{\pm} \cdot \mathbf{n} ]\!] [\![ \nabla v_{\pm} \cdot \mathbf{n} ]\!]    
$$
* The facet sets  $\mathcal{F}^{\pm}$ contain all facets lying between elements of the active mesh corresponding to subdomain $\Omega_{\pm}$. In the sketch given below the facets in  $\mathcal{F}^{-}$ are indicated by red dashed lines. We obtain them below using `GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)`. 
* Obviously, there is a close connection between the 'derivative-jump' version of the ghost-penalty stabilization often employed in unfitted methods and $J_{ \mathrm{CIP} }$. Whereas the former is usually limited to a band of facets around the interface, we employ the latter here globally. On the other hand, it is not necessary to include higher order jumps terms into $J_{ \mathrm{CIP} }$ as is usually done for the derivative-jump' version of the ghost-penalty when `order` is larger one. 

<div>
<img src="graphics/Stab-sketch-notebook.png" width="600"/>
</div>

In [ ]:
ba_facets = { NEG: GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg),
              POS: GetFacetsWithNeighborTypes(mesh, a=haspos, b=haspos) }
dk = tuple([ dCut(lsetp1, dt, skeleton=True, definedonelements=ba_facets[dt], deformation=lsetadap.deform)
               for dt in [NEG,POS] ])
nF = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

gamma_CIP = 5e-2
a += sum( [ gamma_CIP * h * mu[i] * InnerProduct( (gradu[i] - gradu[i].Other()) * nF, 
             (gradv[i] - gradv[i].Other()) * nF ) * dk[i] for i in [0,1] ]  )

## Galerkin-least squares 
Additionally, we add a Galerkin-least-squares stabilization which incorporates the PDE constraint element-wise: 

$$ 
J_{ \mathrm{GLS} }(u,v) := \sum\limits_{\pm} \sum\limits_{ T \in \mathcal{T}^{\pm} } h^2 ( \mathcal{L} u_{\pm} , \mathcal{L} v_{\pm} )_{   T_{\pm}  }
$$

Here, $T^{-} := T \cap \Omega_{-,h} $ is the part of the element lying in domain $\Omega_{-}$ (as indicated in the sketch above in cyan). The differential operator $\mathcal{L}$ is implemented below in the function `calL`.
Since the PDE is inhomogeneous, we also have to add a corresponding term 

$$
\sum\limits_{\pm} \sum\limits_{ T \in \mathcal{T}^{\pm} } h^2 ( f_{\pm} , \mathcal{L} v_{\pm} )_{   T_{\pm}  }
$$

to the right hand side to preserve consistency. Note that this stabilization is only relevant for `order` larger one since piecewise linear functions are in the kernel of $\mathcal{L}$.

In [ ]:
def calL(fun):
    hesse = [fun[i].Operator("hesse") for i in [0,1]]
    return (-mu[0]*hesse[0][0,0]-mu[0]*hesse[0][1,1],-mu[1]*hesse[1][0,0]-mu[1]*hesse[1][1,1])
gamma_GLS = 5e-2
a += sum( [ gamma_GLS * h**2 * calL(u)[i] * calL(v)[i] * dC[i] for i in [0, 1] ] )
f += sum( [ gamma_GLS * h**2 * coef_f[i] * calL(v)[i] * dC[i] for i in [0, 1] ] )

## Stabilization on  the interface 
We use the following stabilization terms on the interfae:

$$
J_{ \mathrm{H} }^{ \Gamma_h }(u,v) := \frac{ \bar{\mu} }{h}  \int\limits_{ \Gamma_h } [\![u]\!] [\![v]\!], \qquad 
J_{ \mathrm{N} }^{ \Gamma_h }(u,v) := h \int\limits_{ \Gamma_h } [\![\mu \nabla u \cdot \mathbf{n}]\!] [\![\mu \nabla v \cdot \mathbf{n}]\!],   \qquad 
J_{ \mathrm{T} }^{ \Gamma_h }(u,v) := h \bar{\mu} \int\limits_{ \Gamma_h } [\![ \nabla_{\Gamma_h } u ]\!] [\![ \nabla_{\Gamma_h } v  ]\!]
$$
Here,
* $J_{ \mathrm{H} }^{ \Gamma_h }$ is simply the stability term in the usual unfitted Nitsche formulation. 
* $J_{ \mathrm{N} }^{ \Gamma_h }(u,v)$ penalizes the normal jumps across the interface 
* and $J_{ \mathrm{T} }^{ \Gamma_h }$ the tangential jump on the interface. Here, we defined the projector $P_{\Gamma_h} := I - \mathbf{n}_{\Gamma_h} (\mathbf{n}_{\Gamma_h})^T$ and the surface gradient $\nabla_{\Gamma_h} := P_{\Gamma_h} \nabla$. 


In [ ]:
def P(fun):
    return fun - (fun * n) * n

jump_flux_u =  (mu[0] * gradu[0] - mu[1] * gradu[1]) * n
jump_flux_v =  (mu[0] * gradv[0] - mu[1] * gradv[1]) * n
jump_tangential_u =  P(gradu[0]) - P(gradu[1])
jump_tangential_v =  P(gradv[0]) - P(gradv[1])

gamma_IF = 1e-3
mubar = (mu[0]+mu[1])/2
a += gamma_IF * (mubar/h) * (u[0] - u[1]) * (v[0] - v[1])  * ds
a += gamma_IF * h * jump_flux_u * jump_flux_v * ds
a += gamma_IF * mubar * h * jump_tangential_u * jump_tangential_v * ds

## (Weak) Tikhonov stabilization 
Whereas the previous stabilization terms are consistent for sufficiently smooth solutions of the PDE, we will now add a weakly inconsistent penalty. By this we mean that the resulting perturbation scales with $h$ and the polynomial order `k` in a way that does not spoil the error estimates (see the paper for details). 

$$
J_{ \alpha }(u,v) := h^{2k} \sum\limits_{\pm}  \alpha_1 ( u_{\pm}, v_{\pm})_{ \Omega_{\pm,h}^{\dagger} } + \alpha_2 (\nabla u_{\pm}, \nabla v_{\pm} )_{ \Omega_{\pm,h}^{\dagger} }.
$$

Here, $\Omega_{i,h}^{\dagger}$ encompasses all elements of the active mesh in the interior subdomain obtained by `ci.GetElementsOfType(HASNEG)`  (the union of the cyan and yellow region in the sketch).

In [ ]:
alpha = [1e-5,1e-2]
dGeom = tuple([ dx(definedonelements=ci.GetElementsOfType(dt))
                     for dt in [HASNEG,HASPOS]])
a += sum( alpha[0] * h**(2*order) * u[i] * v[i] * dGeom[i] for i in [0, 1])
a += sum( alpha[1] * h**(2*order) * grad( u[i]) * grad(v[i]) * dGeom[i] for i in [0, 1])

## Dual stabilization 
For the dual stabilization we make the simple choice 
$$
s^{\ast}_h(z_h,w_h) := (\mu \nabla z_h, \nabla w_h)_{ \Omega }.
$$

In [ ]:
a += sum(-mu[i] * gradz[i] * gradw[i] * dC[i] for i in [0, 1])

## Solve linear system and measure the error 

Let us solve the linear system, measure the $L^2$-error in the target domain and plot the solution.

In [ ]:
a.Assemble()
f.Assemble()
gfu = GridFunction(Vh_Gamma)
gfu.vec.data = a.mat.Inverse(Vh_Gamma.FreeDofs(),inverse= "sparsecholesky"  )* f.vec
err_sqr = sum( [ (gfu.components[i]   - solution[i])**2 * dCut(lsetp1, dt,definedon=mesh.Materials("only_B|omega"),
                deformation=lsetadap.deform, order=2*order)   for i,dt in zip([0, 1],[NEG,POS] ) ] )
l2_err = sqrt(Integrate(err_sqr, mesh))
print("L2-error = ", l2_err)

uh = IfPos(lsetp1, gfu.components[1], gfu.components[0])
deform_graph = CoefficientFunction((lsetadap.deform[0], lsetadap.deform[1], 4*uh))
DrawDC(lsetp1, gfu.components[0], gfu.components[1], mesh, "graph_of_u", deformation=deform_graph, min=0, max=0.25)
DrawDC(lsetp1, (gfu.components[0]-solution[0])**2, (gfu.components[1]-solution[1])**2, mesh, "error",min=1e-8, max=1e-3)

Notice that the error is concentrated near the boundary $\partial \Omega$ where data is unknown. Actually, we have to require that $B \setminus \omega$ does not touch $\partial \Omega$ to obtain practically useful error estimates (see our paper for more details).

## Now it's your turn. 
Here are some suggestions for further experiments: 
* Play around with the contrast `mu`. 
* In practice one usually only has access to noisy data. Try to add some noise to the data `f` and `q` and check how the method performs.
* Is it necessary that the discretization spaces for primal and dual variable have the same order? Maybe it also suffices to have `Vh0 = H1(mesh, order=1, dirichlet="bc_Omega",dgjumps=False)`?
* What happens if the mesh deformation is disabled?
